In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [45]:
query2=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE YEAR=2024
# AND GP='Saudi Arabian Grand Prix'
"""
track3=pd.read_gbq(query2,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3607/3053019962.py:6: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



KeyboardInterrupt: 

In [126]:
track2=track3[(track3["GP"]=='Australian Grand Prix')&(track3["Year"]==2024)]

In [127]:
#Convert lapnumber to string for discrete chart coloring
track2["LapNumber"]=track2["LapNumber"].astype(str)

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3607/1109796388.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [128]:
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [129]:
Drivers=(track2['Driver'].unique())

In [130]:
stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stints.head()

,Driver,Stint,Compound,StintLength
0,ALB,1.0,MEDIUM,6
1,ALB,2.0,HARD,21
2,ALB,3.0,HARD,30
3,ALO,1.0,HARD,17
4,ALO,2.0,MEDIUM,24


In [131]:
fig=px.bar(
    stints,
    y="Driver",
    x='StintLength',
    text="StintLength",
    orientation='h',
    barmode='stack',
    color='Compound',
    template="plotly_dark",
    title="<b>Tyre Strategies for the {} {}</b>".format(year,gp),
    height=800, width=1200,
    color_discrete_map={
                 "SOFT": "#F91536",
                 "MEDIUM": "#ffdf00 ",
                 "HARD":"#c8c8c8 ",
                 "INTERMEDIATE":"#009E60 ",
                 "WET":"#3671C6"
    }
)
fig.update_layout(
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    xaxis_title="<b>Lap Number</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow",
    # font_family="Courier New",
    # font_color="blue",
    # title_font_color="red",
    # legend_title_font_color="green"

#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig.update_layout(yaxis={'categoryorder':'total descending'},    
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),)

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

fig.update_yaxes(ticksuffix = "  ")

fig.show()

In [132]:
fig.write_html("{} {} Tyre Strategies.html".format(year,gp))